# InSituPy demonstration - InSituExperiment

In [1]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from insitupy import read_xenium
import scanpy as sc
from insitupy import InSituExperiment

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Load Xenium data into `InSituData` object

Now the Xenium data can be parsed by providing the data path to the `InSituPy` project folder.

In [4]:
insitupy_project = Path("demo_dataset/demo_insitupy_project")
xd = read_xenium(insitupy_project)
xd.load_all(skip="transcripts")

Loading annotations...
Loading cells...
	No alternative cells found...
Loading images...
['nuclei', 'CD20', 'HER2', 'DAPI', 'HE']
Loading regions...


### Create `InSituExperiment`

#### Method 1: Manually add `InSituData` objects

In [5]:
exp = InSituExperiment()
exp.add(
    data=xd,
    metadata={
        "slide_id": xd.slide_id,
        "sample_id": xd.sample_id,
        "patient": "A"
    }
    )

In [6]:
exp

InSituExperiment with 1 samples:
           uid slide_id    sample_id patient
0     42542689  0001879  Replicate 1       A

In the same way also other datasets can be added. For demonstration purposes, we just add the same dataset again and change the metadata.

In [7]:
exp.add(
    data=xd,
    metadata={
        "slide_id": xd.slide_id,
        "sample_id": xd.sample_id,
        "patient": "A",
        "therapy": "drugB"
    }
    )

In [8]:
exp

InSituExperiment with 2 samples:
           uid slide_id    sample_id patient therapy
0     42542689  0001879  Replicate 1       A     NaN
1     512346d0  0001879  Replicate 1       A   drugB

#### Method 2: From config file

As config file either a csv file or an excel file can be used.

Example of a valid configuration file:

| directory         | experiment_name | date       | patient    |
|-------------------|-----------------|------------|------------|
| /path/to/dataset1 | Experiment 1    | 2023-09-01 | Patient A  |
| /path/to/dataset2 | Experiment 2    | 2023-09-02 | Patient B  |

In [9]:
exp = InSituExperiment.from_config(config_path="./demo_dataset/insituexperiment_config.csv")

In [10]:
exp

InSituExperiment with 2 samples:
           uid slide_id    sample_id patient therapy
0     1191fe9e  0001879  Replicate 1       A   drugA
1     201c2818  0001879  Replicate 1       B   drugB

#### Method 3: From regions

We can also use regions from an `InSituData` object to split the data into separate datasets and create an `InSituExperiment` from them. This can be used to select the most interesting regions and focus on them for the analysis or to split a TMA dataset into separate datasets for each core.

In [11]:
exp = InSituExperiment.from_regions(
    data=xd, region_key="TMA"
)

In [12]:
exp

InSituExperiment with 6 samples:
           uid slide_id    sample_id region_key region_name
0     e6ad994b  0001879  Replicate 1        TMA         A-1
1     1bc49e44  0001879  Replicate 1        TMA         A-2
2     e7cb6fb8  0001879  Replicate 1        TMA         A-3
3     d2c65194  0001879  Replicate 1        TMA         B-1
4     2fefaf1d  0001879  Replicate 1        TMA         B-2
5     e82d138c  0001879  Replicate 1        TMA         B-3

In [13]:
exp.show(1)

Invalid schema for package 'ome-types', please run 'npe2 validate ome-types' to check for manifest errors.


Viewer(camera=Camera(center=(0.0, 499.90625, 499.8), zoom=0.2945552291054573, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 1000.025, 0.2125), (0.0, 999.8125, 0.2125)), current_step=(2352, 2352), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'nuclei' at 0x1a2c158f250>, <Image layer 'CD20' at 0x1a2b703a880>, <Image layer 'HER2' at 0x1a2c6cefaf0>, <Image layer 'DAPI' at 0x1a2c6df8e20>, <Image layer 'HE' at 0x1a2c716ffa0>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='0001879: Replicate 1', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000001A209F70EE0>], _persiste

## Add new metadata

In the following section different scenarios for adding new metadata are shown.

In [13]:
exp1 = InSituExperiment.from_regions(
    data=xd, region_key="demo_regions"
)

In [14]:
exp1

InSituExperiment with 3 samples:
           uid slide_id    sample_id    region_key region_name
0     2c1d7cdd  0001879  Replicate 1  demo_regions     Region1
1     090a2e69  0001879  Replicate 1  demo_regions     Region2
2     c3802d40  0001879  Replicate 1  demo_regions     Region3

In [15]:

exp2 = exp.copy()
exp2.append_metadata(
    new_metadata="./demo_dataset/insituexperiment_new_metadata.csv",
    by="region_name", overwrite=True
)

In [16]:
exp2

InSituExperiment with 6 samples:
           uid slide_id    sample_id region_name therapy organ test region_key
0     e6ad994b  0001879  Replicate 1         A-1     NaN   NaN  NaN        NaN
1     1bc49e44  0001879  Replicate 1         A-2     NaN   NaN  NaN        NaN
2     e7cb6fb8  0001879  Replicate 1         A-3     NaN   NaN  NaN        NaN
3     d2c65194  0001879  Replicate 1         B-1     NaN   NaN  NaN        NaN
4     2fefaf1d  0001879  Replicate 1         B-2     NaN   NaN  NaN        NaN
5     e82d138c  0001879  Replicate 1         B-3     NaN   NaN  NaN        NaN

In [17]:

exp3 = exp.copy()
exp3.append_metadata(
    new_metadata="./demo_dataset/insituexperiment_new_metadata.csv",
    by="region_name", overwrite=False
)

In [18]:
exp3

InSituExperiment with 6 samples:
           uid slide_id    sample_id region_key region_name organ test therapy
0     e6ad994b  0001879  Replicate 1        TMA         A-1   NaN  NaN     NaN
1     1bc49e44  0001879  Replicate 1        TMA         A-2   NaN  NaN     NaN
2     e7cb6fb8  0001879  Replicate 1        TMA         A-3   NaN  NaN     NaN
3     d2c65194  0001879  Replicate 1        TMA         B-1   NaN  NaN     NaN
4     2fefaf1d  0001879  Replicate 1        TMA         B-2   NaN  NaN     NaN
5     e82d138c  0001879  Replicate 1        TMA         B-3   NaN  NaN     NaN

In [19]:

exp4 = exp.copy()
exp4.append_metadata(
    new_metadata="./demo_dataset/insituexperiment_new_metadata2.csv",
    by="region_name", overwrite=False
)

In [20]:
exp4

InSituExperiment with 6 samples:
           uid slide_id    sample_id region_key region_name organ test therapy
0     e6ad994b  0001879  Replicate 1        TMA         A-1   NaN  NaN     NaN
1     1bc49e44  0001879  Replicate 1        TMA         A-2   NaN  NaN     NaN
2     e7cb6fb8  0001879  Replicate 1        TMA         A-3   NaN  NaN     NaN
3     d2c65194  0001879  Replicate 1        TMA         B-1   NaN  NaN     NaN
4     2fefaf1d  0001879  Replicate 1        TMA         B-2   NaN  NaN     NaN
5     e82d138c  0001879  Replicate 1        TMA         B-3   NaN  NaN     NaN

In [21]:

exp5 = exp.copy()
exp5.append_metadata(
    new_metadata="./demo_dataset/insituexperiment_new_metadata2.csv",
    by="region_name", overwrite=True
)

In [22]:
exp5

InSituExperiment with 6 samples:
           uid slide_id    sample_id region_name therapy organ test region_key
0     e6ad994b  0001879  Replicate 1         A-1     NaN   NaN  NaN        NaN
1     1bc49e44  0001879  Replicate 1         A-2     NaN   NaN  NaN        NaN
2     e7cb6fb8  0001879  Replicate 1         A-3     NaN   NaN  NaN        NaN
3     d2c65194  0001879  Replicate 1         B-1     NaN   NaN  NaN        NaN
4     2fefaf1d  0001879  Replicate 1         B-2     NaN   NaN  NaN        NaN
5     e82d138c  0001879  Replicate 1         B-3     NaN   NaN  NaN        NaN

## Concatenate multiple `InSituExperiment` objects

In [23]:
exp

InSituExperiment with 6 samples:
           uid slide_id    sample_id region_key region_name
0     e6ad994b  0001879  Replicate 1        TMA         A-1
1     1bc49e44  0001879  Replicate 1        TMA         A-2
2     e7cb6fb8  0001879  Replicate 1        TMA         A-3
3     d2c65194  0001879  Replicate 1        TMA         B-1
4     2fefaf1d  0001879  Replicate 1        TMA         B-2
5     e82d138c  0001879  Replicate 1        TMA         B-3

In [24]:
exp1

InSituExperiment with 3 samples:
           uid slide_id    sample_id    region_key region_name
0     2c1d7cdd  0001879  Replicate 1  demo_regions     Region1
1     090a2e69  0001879  Replicate 1  demo_regions     Region2
2     c3802d40  0001879  Replicate 1  demo_regions     Region3

In [25]:
exp_concat = InSituExperiment.concat(
    objs={
        "exp_TMA": exp,
        "exp_demo_regions": exp1
    },
    new_col_name="name"
    )

In [26]:
exp_concat

InSituExperiment with 9 samples:
           uid slide_id    sample_id    region_key region_name              name
0     e6ad994b  0001879  Replicate 1           TMA         A-1           exp_TMA
1     1bc49e44  0001879  Replicate 1           TMA         A-2           exp_TMA
2     e7cb6fb8  0001879  Replicate 1           TMA         A-3           exp_TMA
3     d2c65194  0001879  Replicate 1           TMA         B-1           exp_TMA
4     2fefaf1d  0001879  Replicate 1           TMA         B-2           exp_TMA
5     e82d138c  0001879  Replicate 1           TMA         B-3           exp_TMA
6     2c1d7cdd  0001879  Replicate 1  demo_regions     Region1  exp_demo_regions
7     090a2e69  0001879  Replicate 1  demo_regions     Region2  exp_demo_regions
8     c3802d40  0001879  Replicate 1  demo_regions     Region3  exp_demo_regions

In [27]:
exp_concat = InSituExperiment.concat(
    objs=[exp, exp1])

In [28]:
exp_concat

InSituExperiment with 9 samples:
           uid slide_id    sample_id    region_key region_name
0     e6ad994b  0001879  Replicate 1           TMA         A-1
1     1bc49e44  0001879  Replicate 1           TMA         A-2
2     e7cb6fb8  0001879  Replicate 1           TMA         A-3
3     d2c65194  0001879  Replicate 1           TMA         B-1
4     2fefaf1d  0001879  Replicate 1           TMA         B-2
5     e82d138c  0001879  Replicate 1           TMA         B-3
6     2c1d7cdd  0001879  Replicate 1  demo_regions     Region1
7     090a2e69  0001879  Replicate 1  demo_regions     Region2
8     c3802d40  0001879  Replicate 1  demo_regions     Region3

## Differential gene expression analysis

In [29]:
from insitupy import differential_gene_expression

In [33]:
exp = InSituExperiment.from_regions(
    data=xd, region_key="demo_regions"
)

In [34]:
exp

InSituExperiment with 3 samples:
           uid slide_id    sample_id    region_key region_name
0     c22613af  0001879  Replicate 1  demo_regions     Region1
1     d660fdff  0001879  Replicate 1  demo_regions     Region2
2     ce9edfb5  0001879  Replicate 1  demo_regions     Region3

In [37]:
exp.show(2)

Viewer(camera=Camera(center=(0.0, 669.1624999999999, 638.3499999999999), zoom=0.21649878123990487, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 1338.5375, 0.2125), (0.0, 1276.9125, 0.2125)), current_step=(3149, 3004), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'nuclei' at 0x22b3ad4b670>, <Image layer 'CD20' at 0x22b3ad6ceb0>, <Image layer 'HER2' at 0x22b3bfbb880>, <Image layer 'DAPI' at 0x22b377c1df0>, <Image layer 'HE' at 0x22b3e0b41f0>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='0001879: Replicate 1', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000002

In [39]:
exp.data[2].store_geometries()

Added 2 new annotations to key 'Test'


In [46]:
exp.data[2].annotations.save(r"C:\Users\ge37voy\Downloads\annotations_out")

In [31]:
exp.data

[InSituData
 Method:		Xenium
 Slide ID:	0001879
 Sample ID:	Replicate 1
 Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
 Metadata file:	.ispy
     ➤ images
        nuclei:	(4706, 4706)
        CD20:	(4706, 4706)
        HER2:	(4706, 4706)
        DAPI:	(4706, 4706)
        HE:	(4706, 4706, 3)
     ➤ cells
        matrix
            AnnData object with n_obs × n_vars = 6743 × 313
            obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'n_genes', 'leiden', 'annotations-TestKey', 'annotations-demo', 'annotations-demo2', 'annotations-demo3', 'cell_type'
            var: 'gene_ids', 'feature_types', 'genome'
            uns: 'counts_location', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'umap'
            obsm: 'OT', 'X_pca', 'X_umap', 'spatial'
            varm: 'OT', 'PCs', 'binned_expression'
            layers: 'counts', 'norm_counts'
            obsp: 'connectivities',